Zde hledáme Wikidata IDs toho autorstva, které je nemá vyplněné v databázi autorit.

In [2]:
import gc
import os
import re
import json
import requests
import pandas as pd

In [3]:
kam = 'data_raw/wikidata_doplneni'
if not os.path.exists(kam):
    os.makedirs(kam)

In [4]:
vsechny_jsony = [f for f in os.listdir(kam)]

In [5]:
vyhledane = set([x.split("_")[0] for x in vsechny_jsony])

In [6]:
df = pd.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))

In [7]:
df = df.explode('100_7')

In [8]:
df['100_7'].nunique()

171983

In [9]:
df.groupby('100_7').size().nlargest(1000)

100_7
jx20060515016    2984
jx20050830020    2373
jn20001103529    1662
jn19981000826    1477
jk01120043       1386
                 ... 
jk01131204         65
jk01140140         65
jk01150131         65
jn19981001994      65
jn19981002320      65
Length: 1000, dtype: int64

In [10]:
opakovane = df.groupby('100_7').size()
opakovane = opakovane.sort_values(ascending=False)
opakovane = opakovane[opakovane > 1].index.to_list()
len(opakovane)

90945

In [11]:
autority = pd.read_parquet(os.path.join("data","aut_vyber.parquet"))

In [12]:
autority

,024_2,024_a,046_f,046_g,100_7,100_a,100_d,100_ind1,110_a,370_a,...,411_a,430_a,450_a,500_a,500_i,500_ind1,550_7,678_a,856_u,leader
001,,,,,,,,,,,,,,,,,,,,,
jk01010001,None,None,[1931],None,[jk01010001],"[Abraham, Jiří,]",[1931-],[1],None,None,...,None,None,None,None,None,None,None,"[Narozen 14.12.1931 v Osvračíně. MUDr., odborn...",None,nz a22 n 4500
jk01010002,None,None,[1890],[1933],[jk01010002],"[Abraham, Josef,]",[1890-1933],[1],None,None,...,None,None,None,None,None,None,None,[Narozen 19.11.1890 v Osvračíně u Horšova Týna...,None,nz a22 n 4500
jk01010003,None,None,[1957],None,[jk01010003],"[Abraham, Pavel,]",[1957-],[1],None,None,...,None,None,None,None,None,None,None,[Narozen roku 1957. Student filosofické fakult...,None,nz a22 n 4500
jk01010004,None,None,[1889],[1963],[jk01010004],"[Absolon, Josef,]",[1889-1963],[1],None,"[Odrlice, Senice na Hané, Česko]",...,None,None,None,None,None,None,None,"[Narozen 27. 1. 1889 v Odrlicích na Moravě, ze...",None,cz a22 n 4500
jk01010005,"[isni, wikidata]","[0000000109542912, Q1063371]",[1877],[1960],[jk01010005],"[Absolon, Karel,]",[1877-1960],[1],None,None,...,None,None,None,None,None,None,None,"[Narozen 16.6.1877 v Boskovicích, zemřel 6.10....",None,cz a22 n 4500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xx0326208,"[isni, wikidata]","[0000000108823165, Q7347168]",[1934],[2018],[xx0326208],"[O'Neil, Robert M.,]",[1934-2018],[1],None,None,...,None,None,None,None,None,None,None,[Americký právník a vysokoškolský pedagog zamě...,None,nz a22 n 4500
xx0326209,"[isni, wikidata]","[0000000115709037, Q6969766]",[1888],[1976],[xx0326209],"[Micklem, Nathaniel,]",[1888-1976],[1],None,None,...,None,None,None,None,None,None,None,"[Britský křesťanský teolog, kongregacionalisti...",None,nz a22 n 4500
xx0326210,"[isni, wikidata]","[000000010928102X, Q45117113]",[1960],[2016],[xx0326210],"[Menniti Ippolito, Antonio,]",[1960-2016],[1],None,None,...,None,None,None,None,None,None,None,[Italský historik a vysokoškolský pedagog mode...,None,nz a22 n 4500


In [13]:
autority = autority[autority['024_2'].notnull()]
autority = autority[autority['024_a'].notnull()]

In [14]:
autority = autority.explode(['024_a','024_2'])

In [15]:
autority_s_wikidaty = set(autority[autority['024_2'] == 'wikidata']['024_a'].index.to_list())

In [16]:
chybi = [x for x in opakovane if x not in autority_s_wikidaty]

In [17]:
del df
del autority
gc.collect()

0

In [18]:
len(chybi)

82412

In [19]:
# Wikidata SPARQL endpoint
sparql_endpoint = "https://query.wikidata.org/sparql"

# Function to get Wikidata ID from NL CR AUT ID
def get_wikidata_id(nl_cr_aut_id):
    query = f"""
    SELECT ?item WHERE {{
      ?item wdt:P691 "{nl_cr_aut_id}".
    }}
    """
    headers = {
        "Accept": "application/sparql-results+json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = requests.get(sparql_endpoint, params={"query": query}, headers=headers)
    data = response.json()

    if "results" in data and "bindings" in data["results"]:
        bindings = data["results"]["bindings"]
        if bindings:
            return bindings[0]["item"]["value"].split("/")[-1]
    return None

In [20]:
vyhledane = set(vyhledane)

In [21]:
co_stahovat = [x for x in chybi if x not in vyhledane]

In [22]:
len(co_stahovat)

287

In [23]:
pocitadlo = 0
for nl_cr_aut_id in co_stahovat:
    pocitadlo += 1
    print(f"{pocitadlo:005}: {nl_cr_aut_id}")
    slovnik = {}
    slovnik['100_7'] = nl_cr_aut_id
    wikidata_id = get_wikidata_id(nl_cr_aut_id)
    if wikidata_id:
        slovnik['024_a'] = wikidata_id
    else:
        slovnik['024_a'] = ""
    with open(os.path.join(kam, f"{nl_cr_aut_id}_{slovnik['024_a']}.json"), "w+", encoding="utf-8") as file:
        file.write(json.dumps(slovnik))

00001: jo20241241000
00002: xx0313952
00003: jo2012704868
00004: jo2016903205
00005: xx0320589
00006: mzk2015880860
00007: xx0134784
00008: mzk2012736469
00009: hka20231189105
00010: hka20241240408
00011: jk01041585
00012: jo20241243733
00013: mzk20241242528
00014: jo20241241479
00015: jo20241239780
00016: mzk20201095020
00017: mzk20221168625
00018: jo2008413902
00019: jo2006324609
00020: xx0083079
00021: jx20131218013
00022: jo2012691518
00023: jo20191022545
00024: xx0100332
00025: jx20050801031
00026: mzk2011673122
00027: xx0280735
00028: jn20000402279
00029: jn20001103098
00030: mzk2013782659
00031: mzk2003181770
00032: mzk2010602479
00033: mzk2003171527
00034: xx0034658
00035: xx0001833
00036: xx0042842
00037: xx0065651
00038: mzk2007423984
00039: xx0011793
00040: mzk2002110164
00041: xx0010244
00042: mub2016898839
00043: mzk2009544356
00044: xx0036309
00045: xx0067403
00046: kv2016926516
00047: xx0036001
00048: mzk2008442871
00049: mzk2008434270
00050: mub2010576985
00051: jx20080